![](SCEA_Logo.png)

# County by County COVID-19 Activity Monitor

The plot below projects reported case data onto axes of prevalence (per capita current cases) and transmittance (week-over-week change in new cases).  Counties that are above 0 on the x-axis have an upward trend in new cases.  Counties that appear in the upper-right quadrant have relatively high prevalence and transmittance, and are especially at-risk.

<br>

The data is updated daily, with each refresh pulling from 3 days prior.  This is an effort to mitigate reporting delays that may be present at the county level.

<br>

The chart is interactive.  Hovering over a datapoint gives more detail about the specific county.  Brushing a region on the left pane by clicking and dragging adds the selected counties to the bar chart in the right pane.

<hr>

In [1]:
from IPython.core.display import display, HTML
import altair as alt

In [2]:
# HTML to remove grey border after nbconvert
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; }' 
        '.container { width:100% !important; }'
        '.end_space { min-height:0px !important; }'
        '.output { align-items: center !important; }'
        '.output_area { width:80%; }'
    '</style>'
))

In [3]:
%run -i 'case_load_vs_case_delta_etl.py'

In [6]:
brush = alt.selection_interval(empty = 'none')

base = alt.Chart(cases_percap_recent.query('wkly_diff < 3 and wkly_diff > -3 and cases_new_per1k_sum14 > 0'))

cases_vs_case_delta_filter = (base.mark_point()
 .encode(x = alt.X('wkly_diff', axis = alt.Axis(title = 'Week-over-week Proportional Change')),
         y = alt.Y('cases_new_per1k_sum14', axis = alt.Axis(title = 'Current Cases per 1000')),
         color = alt.Color('region', legend = alt.Legend(title = 'Region')),
         size = alt.Size('Population', legend = alt.Legend(title = 'Population')),
         tooltip = ['County', 'Current Cases', 'Population'])
 #.interactive()
 .add_selection(brush)
 .properties(height = 400,
             width = 450,
             title = ['Estimated Current Case Load vs Weekly Change in New Cases',
                      'US counties > 100,000 residents'])
)

king = (alt.Chart(cases_percap_recent.query('County == "King, Washington"'))
 .mark_text(dx = 30, text = '- King', color = 'black', font = 'Helvetica', filled = True, size = 20)
 .encode(x = alt.X('wkly_diff', axis = alt.Axis(title = 'Week-over-week Proportional Change')),
         y = alt.Y('cases_new_per1k_sum14', axis = alt.Axis(title = 'Current Cases per 1000'))
        )
)

cases_bar = (base.mark_bar(opacity = .5)
 .encode(x = alt.X('cases_new_per1k_sum14:Q', axis = alt.Axis(title = 'Current Cases per 1000')),
         y = alt.Y('County:N',
                   axis = alt.Axis(labels = False),
                   sort = alt.EncodingSortField(field='cases_new_per1k_sum14', order='descending')),
         color = alt.Color('region')
         )
 .transform_filter(brush)
 .properties(height = 400, 
             width = 250,
             title = 'Selected County Prevalence')
)

labels = (cases_bar.mark_text(align = 'right', baseline = 'middle')
          .encode(text = 'County:N')

)

(cases_vs_case_delta_filter + king) | (cases_bar + labels).resolve_scale(y = 'independent')

alt.HConcatChart(...)

In [5]:
print('Data Updated: ' + max(cases_percap_recent['date']).strftime('%Y-%m-%d'))

Data Updated: 2020-08-22


<hr>
<br>

### Definitions
Current Cases per 1000: 14-day rolling-sum of new cases, divided by the 2017 US Census Bureau estimate  

Week-over-week Proportional Change: Total new cases from last week minus total new cases from two weeks ago, divided by total new cases from two weeks ago

### COVID-19 Data Source
[New York Times](https://github.com/nytimes/covid-19-data)


### Code
[GitHub](https://github.com/amcadie/covid_19_hospitalization_estimate)